In [ ]:
# Install COMET

pip install unbabel-comet

In [ ]:
# In case of an error in importing transformers run these two lines of code

pip uninstall transformers -y

Found existing installation: transformers 4.50.3
Uninstalling transformers-4.50.3:
  Successfully uninstalled transformers-4.50.3


In [ ]:
# Reinstall

pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 54.7 MB/s eta 0:00:00


In [ ]:
# Code for translation of the first 5 chapters of the old testametn

import xml.etree.ElementTree as ET
from tqdm import tqdm
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from comet import download_model, load_from_checkpoint
import pandas as pd
import sacrebleu

# M2M100
model_name = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name)
tokenizer.src_lang = "ru"  # Source language

# COMET
comet_model_path = download_model("Unbabel/wmt22-comet-da")
comet = load_from_checkpoint(comet_model_path)

# Load alignment and language files
alignment_file = 'en-ru.xml' # Alignment file
english_file = 'English.xml' # Target language file
dutch_file = 'Russian.xml' # Source language file

alignment_root = ET.parse(alignment_file).getroot()
english_root = ET.parse(english_file).getroot()
dutch_root = ET.parse(dutch_file).getroot()

# Find segements with the id
def find_segment(root, seg_id):
    for seg in root.findall(".//s"):
        if seg.get('id') == seg_id:
            sentence_number = seg.get('id')
            text = ' '.join([w.text for w in seg.findall('w') if w.text])
            return text, sentence_number if sentence_number else 'N/A'
    return None, 'N/A'

# Initialize score storage
results = []
chapter_scores = []
current_chapter = None

# Get alignment from file
links = alignment_root.findall(".//link")

# Loop trough segments and use tqdm to view progress
for link in tqdm(links, desc="Processing links"):
    xtargets = link.get('xtargets').split(';')
    if len(xtargets) != 2:
        continue

    en_ids = xtargets[0].split()
    nl_ids = xtargets[1].split()

    ref_en, en_number = zip(*[find_segment(english_root, en_id) for en_id in en_ids])
    ref_en = " ".join(filter(None, ref_en))

    src_nl, nl_number = zip(*[find_segment(dutch_root, nl_id) for nl_id in nl_ids])
    src_nl = " ".join(filter(None, src_nl))

    # Sentence number format 'b.GEN.#.#.#'
    chapter = en_ids[0].split('.')[2] if en_ids else 'Unknown'

    # Stop if chapter is bigger than 5
    if chapter.isdigit() and int(chapter) > 5:
        break

    # Check chapter end and append data summary
    if chapter != current_chapter:
        if current_chapter is not None:
            chapter_df = pd.DataFrame(chapter_scores)
            avg_comet = chapter_df["COMET Score"].mean()
            avg_bleu = chapter_df["SacreBLEU Score"].mean()
            avg_chrf = chapter_df["CHRF Score"].mean()
            results.append({
                "Dutch Source": "Chapter Average:",
                "Reference English": current_chapter,
                "Translated English": "",
                "COMET Score": round(avg_comet, 4),
                "SacreBLEU Score": round(avg_bleu, 2),
                "CHRF Score": round(avg_chrf, 2),
                "English Sentence Numbers": "",
                "Dutch Sentence Numbers": ""
            })
        current_chapter = chapter
        chapter_scores = []

    if not ref_en or not src_nl:
        continue

    # Translation
    inputs = tokenizer(src_nl, return_tensors="pt")
    translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.get_lang_id("en"))
    translated_en = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

    # COMET
    comet_input = [{
        "src": src_nl,
        "mt": translated_en,
        "ref": ref_en
    }]
    score = comet.predict(comet_input, batch_size=1)
    comet_score = score.system_score

    # SacreBLEU
    bleu = sacrebleu.sentence_bleu(translated_en, [ref_en])
    bleu_score = bleu.score

    # CHRF
    chrf = sacrebleu.sentence_chrf(translated_en, [ref_en])
    chrf_score = chrf.score

    chapter_scores.append({
        "COMET Score": comet_score,
        "SacreBLEU Score": bleu_score,
        "CHRF Score": chrf_score
    })

    print({
        "English Sentence Numbers": ", ".join(en_ids),
        "Dutch Sentence Numbers": ", ".join(nl_ids),
        "Dutch Source": src_nl,
        "Reference English": ref_en,
        "Translated English": translated_en,
        "COMET Score": round(comet_score, 4),
        "SacreBLEU Score": round(bleu_score, 2),
        "CHRF Score": round(chrf_score, 2)
    })

    # Append to results
    results.append({
        "English Sentence Numbers": ", ".join(en_ids),
        "Dutch Sentence Numbers": ", ".join(nl_ids),
        "Dutch Source": src_nl,
        "Reference English": ref_en,
        "Translated English": translated_en,
        "COMET Score": round(comet_score, 4),
        "SacreBLEU Score": round(bleu_score, 2),
        "CHRF Score": round(chrf_score, 2)
    })

# Save results to CSV
df = pd.DataFrame(results)
df.to_csv("translation_comet_bleu_chrf_results_first_5_chapters.csv", index=False)
print("Saved")


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
Processing links:   0%|          | 0/62202 [00:00<?, ?it/s]INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilitie

{'English Sentence Numbers': 'b.GEN.1.1.1', 'Dutch Sentence Numbers': 'b.GEN.1.1.1', 'Dutch Source': 'В начале сотворил Бог небо и землю .', 'Reference English': 'In the beginning God created the heaven and the earth .', 'Translated English': 'In the beginning God created heaven and earth.', 'COMET Score': 0.9306, 'SacreBLEU Score': 45.81, 'CHRF Score': 75.89}


INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
Processing links:   0%|          | 2/62202 [00:36<309:39:18, 17.92s/it]

{'English Sentence Numbers': 'b.GEN.1.2.1, b.GEN.1.2.2, b.GEN.1.2.3', 'Dutch Sentence Numbers': 'b.GEN.1.2.1', 'Dutch Source': 'Земля же была безвидна и пуста , и тьма над бездною , и Дух Божий носился над водою .', 'Reference English': 'And the earth was without form , and void ; and darkness was upon the face of the deep . And the Spirit of God moved upon the face of the waters .', 'Translated English': 'But the earth was invisible and empty, and darkness over the desert, and the Spirit of God was over the water.', 'COMET Score': 0.7758, 'SacreBLEU Score': 12.88, 'CHRF Score': 42.4}


...

{'English Sentence Numbers': 'b.GEN.5.31.1', 'Dutch Sentence Numbers': 'b.GEN.5.31.1', 'Dutch Source': 'Всех же дней Ламеха было семьсот семьдесят семь лет ; и он умер .', 'Reference English': 'And all the days of Lamech were seven hundred seventy and seven years : and he died .', 'Translated English': 'All the days of Lamech were seven hundred and seventy years, and he died.', 'COMET Score': 0.7739, 'SacreBLEU Score': 49.75, 'CHRF Score': 75.39}


INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
Processing links:   0%|          | 138/62202 [33:36<213:27:40, 12.38s/it]

{'English Sentence Numbers': 'b.GEN.5.32.1', 'Dutch Sentence Numbers': 'b.GEN.5.32.1', 'Dutch Source': 'Ною было пятьсот лет и родил Ной Сима , Хама и Иафета .', 'Reference English': 'And Noah was five hundred years old : and Noah begat Shem , Ham , and Japheth .', 'Translated English': 'Noah was five hundred years old, and Noah gave birth to Sima, Hama and Yafet.', 'COMET Score': 0.7157, 'SacreBLEU Score': 34.23, 'CHRF Score': 56.26}


Processing links:   0%|          | 138/62202 [33:36<251:57:29, 14.61s/it]

✅ Saved to translation_comet_bleu_chrf_results_first_5_chapters.csv


In [ ]:
# Code for translation of the first 5 chapters of the new testametn

import xml.etree.ElementTree as ET
from tqdm import tqdm
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from comet import download_model, load_from_checkpoint
import pandas as pd
import sacrebleu

# M2M100
model_name = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name)
tokenizer.src_lang = "ru"  # Source language

# COMET
comet_model_path = download_model("Unbabel/wmt22-comet-da")
comet = load_from_checkpoint(comet_model_path)

# Alignment and language files
alignment_file = 'en-ru.xml' # Alignment file
english_file = 'English.xml' # Target language file
dutch_file = 'Russian.xml' # Source language file

alignment_root = ET.parse(alignment_file).getroot()
english_root = ET.parse(english_file).getroot()
dutch_root = ET.parse(dutch_file).getroot()

# Find segements with the id
def find_segment(root, seg_id):
    for seg in root.findall(".//s"):
        if seg.get('id') == seg_id:
            sentence_number = seg.get('id')
            text = ' '.join([w.text for w in seg.findall('w') if w.text])
            return text, sentence_number if sentence_number else 'N/A'
    return None, 'N/A'

# Initialize score storage
results = []
chapter_scores = []
current_chapter = None

# Get alignment links between L23146 to L23283 (Start and end of Matthew 1 New Testament)
links = alignment_root.findall(".//link")

# Loop trough segments and use tqdm to view progress
processing = False
for link in tqdm(links):
    link_id = link.get('id')

    # Start from L23146 (start of the new testament Matthew 1)
    if link_id == "L23146":
        processing = True

    if not processing:
        continue

    xtargets = link.get('xtargets').split(';')
    if len(xtargets) != 2:
        continue

    en_ids = xtargets[0].split()
    nl_ids = xtargets[1].split()

    ref_en, en_number = zip(*[find_segment(english_root, en_id) for en_id in en_ids])
    ref_en = " ".join(filter(None, ref_en))

    src_nl, nl_number = zip(*[find_segment(dutch_root, nl_id) for nl_id in nl_ids])
    src_nl = " ".join(filter(None, src_nl))

    # Determine the chapter from the sentence number
    chapter = en_ids[0].split('.')[1] if en_ids else 'Unknown'

    # Check chapter end and append data summary
    if chapter != current_chapter:
        if current_chapter is not None:
            chapter_df = pd.DataFrame(chapter_scores)
            avg_comet = chapter_df["COMET Score"].mean()
            avg_bleu = chapter_df["SacreBLEU Score"].mean()
            avg_chrf = chapter_df["CHRF Score"].mean()
            results.append({
                "Dutch Source": "Chapter Average:",
                "Reference English": current_chapter,
                "Translated English": "",
                "COMET Score": round(avg_comet, 4),
                "SacreBLEU Score": round(avg_bleu, 2),
                "CHRF Score": round(avg_chrf, 2),
                "English Sentence Numbers": "",
                "Dutch Sentence Numbers": ""
            })
        current_chapter = chapter
        chapter_scores = []

    if not ref_en or not src_nl:
        continue

    # Translation
    inputs = tokenizer(src_nl, return_tensors="pt")
    translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.get_lang_id("en"))
    translated_en = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

    # COMET
    comet_input = [{
        "src": src_nl,
        "mt": translated_en,
        "ref": ref_en
    }]
    score = comet.predict(comet_input, batch_size=1)
    comet_score = score.system_score

    # SacreBLEU
    bleu = sacrebleu.sentence_bleu(translated_en, [ref_en])
    bleu_score = bleu.score

    # CHRF++
    chrf = sacrebleu.sentence_chrf(translated_en, [ref_en])
    chrf_score = chrf.score

    chapter_scores.append({
        "COMET Score": comet_score,
        "SacreBLEU Score": bleu_score,
        "CHRF Score": chrf_score
    })

    print({
        "English Sentence Numbers": ", ".join(en_ids),
        "Dutch Sentence Numbers": ", ".join(nl_ids),
        "Dutch Source": src_nl,
        "Reference English": ref_en,
        "Translated English": translated_en,
        "COMET Score": round(comet_score, 4),
        "SacreBLEU Score": round(bleu_score, 2),
        "CHRF Score": round(chrf_score, 2)
    })

    # Append to results
    results.append({
        "English Sentence Numbers": ", ".join(en_ids),
        "Dutch Sentence Numbers": ", ".join(nl_ids),
        "Dutch Source": src_nl,
        "Reference English": ref_en,
        "Translated English": translated_en,
        "COMET Score": round(comet_score, 4),
        "SacreBLEU Score": round(bleu_score, 2),
        "CHRF Score": round(chrf_score, 2)
    })

    # Stop after L23283 (End of Matthew New Testament)
    if link_id == "L23283":
        break

# Save the results to CSV
df = pd.DataFrame(results)
df.to_csv("translation_comet_bleu_chrf_results_MAT_1_to_5.csv", index=False)
print("Saved")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
Processing links:   0%|          | 0/62202 [00:00<?, ?it/s]INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilitie

{'English Sentence Numbers': 'b.MAT.1.1.1', 'Dutch Sentence Numbers': 'b.MAT.1.1.1', 'Dutch Source': 'Родословие Иисуса Христа , Сына Давидова , Сына Авраамова .', 'Reference English': 'The book of the generation of Jesus Christ , the son of David , the son of Abraham .', 'Translated English': 'The family of Jesus Christ, the Son of David, the Son of Abraham.', 'COMET Score': 0.7626, 'SacreBLEU Score': 38.85, 'CHRF Score': 58.84}


Processing links:  37%|███▋      | 23145/62202 [00:40<00:46, 841.01it/s]INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
Processing links:  37%|███▋      | 23146/62202 [00:45<01:29, 437.91it/s]

{'English Sentence Numbers': 'b.MAT.1.2.1, b.MAT.1.2.2, b.MAT.1.2.3', 'Dutch Sentence Numbers': 'b.MAT.1.2.1', 'Dutch Source': 'Авраам родил Исаака ; Исаак родил Иакова ; Иаков родил Иуду и братьев его ;', 'Reference English': 'Abraham begat Isaac ; and Isaac begat Jacob ; and Jacob begat Judas and his brethren ;', 'Translated English': 'Abraham gave birth to Isaac, Isaac gave birth to Jacob, and Jacob gave birth to Judah and his brothers.', 'COMET Score': 0.737, 'SacreBLEU Score': 5.81, 'CHRF Score': 40.1}


...

{'English Sentence Numbers': 'b.MAT.5.46.1, b.MAT.5.46.2', 'Dutch Sentence Numbers': 'b.MAT.5.46.1, b.MAT.5.46.2', 'Dutch Source': 'Ибо если вы будете любить любящих вас , какая вам награда ? Не то же ли делают и мытари ?', 'Reference English': 'For if ye love them which love you , what reward have ye ? do not even the publicans the same ?', 'Translated English': 'For if you love those who love you, what is your reward?', 'COMET Score': 0.5833, 'SacreBLEU Score': 13.21, 'CHRF Score': 33.13}


INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
Processing links:  37%|███▋      | 23281/62202 [32:24<137:21:21, 12.70s/it]

{'English Sentence Numbers': 'b.MAT.5.47.1, b.MAT.5.47.2', 'Dutch Sentence Numbers': 'b.MAT.5.47.1, b.MAT.5.47.2', 'Dutch Source': 'И если вы приветствуете только братьев ваших , что особенного делаете ? Не так же ли поступают и язычники ?', 'Reference English': 'And if ye salute your brethren only , what do ye more than others ? do not even the publicans so ?', 'Translated English': 'And if you only welcome your brothers, what special do you do?', 'COMET Score': 0.5153, 'SacreBLEU Score': 5.56, 'CHRF Score': 28.62}


INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
Processing links:  37%|███▋      | 23281/62202 [32:32<54:23, 11.93it/s]    

{'English Sentence Numbers': 'b.MAT.5.48.1', 'Dutch Sentence Numbers': 'b.MAT.5.48.1', 'Dutch Source': 'Итак будьте совершенны , как совершен Отец ваш Небесный .', 'Reference English': 'Be ye therefore perfect , even as your Father which is in heaven is perfect .', 'Translated English': 'So be perfect, as your Heavenly Father is perfect.', 'COMET Score': 0.7656, 'SacreBLEU Score': 13.46, 'CHRF Score': 42.48}
✅ Saved to translation_comet_bleu_chrf_results_MAT_1_to_5.csv
